# 🔍 Agentic RAG with Azure AI Foundry (Python)

## 📋 Learning Objectives

This notebook demonstrates how to build intelligent Retrieval-Augmented Generation (RAG) systems using the Microsoft Agent Framework with Azure AI Foundry. You'll learn to create agents that can search through documents and provide accurate, grounded responses based on uploaded content.

**Key RAG Capabilities You'll Build:**
- 📚 **Document Upload**: Upload and process documents for agent access
- 🔍 **Vector Store Creation**: Create searchable vector stores for semantic retrieval
- 🤖 **Grounded Responses**: Build agents that only answer from document content
- 🛡️ **Source Attribution**: Ensure responses are traceable to source documents

## 🎯 RAG Architecture Concepts

### Core RAG Components
- **Document Processing**: Upload files to Azure AI Foundry for indexing
- **Vector Store**: Semantic search capability for relevant content retrieval
- **File Search Tool**: Agent tool that queries the vector store
- **Grounded Generation**: AI responses based only on retrieved context

### Technical Flow
```python
Document Upload → Vector Store Creation → Agent Configuration
                        ↓                       ↓
User Query → File Search Tool → Retrieved Context → Grounded Response
```

## 🏗️ Technical Architecture

### Azure AI Foundry Integration
- **AIProjectClient**: Manages Azure AI project resources and operations
- **AzureAIAgentClient**: Provides agent functionality with Azure backend
- **Vector Store Management**: Handles document indexing and retrieval
- **File Search Tool**: Built-in tool for semantic document search

### Agent Behavior
- Only answers questions using information from uploaded documents
- Clearly indicates when questions cannot be answered from available content
- Provides explicit citations to source documents
- Avoids general knowledge responses outside document scope

## ⚙️ Prerequisites & Setup

**Required Dependencies:**
```bash
pip install agent-framework-core azure-ai-projects azure-identity -U
```

**Azure Configuration:**
- Azure subscription with AI Foundry access
- Azure CLI authentication configured
- Environment variables in `.env` file:
  ```env
  AZURE_AI_PROJECT_ENDPOINT=your_project_endpoint
  AZURE_AI_MODEL_DEPLOYMENT_NAME=your_model_deployment
  ```

**Authentication:**
```bash
az login --tenant <Your Azure Tenant>
```

## 📊 Use Cases

### Knowledge Management
- Corporate document Q&A systems
- Policy and procedure guidance
- Technical documentation assistants
- Training material search

### Customer Support
- Product documentation queries
- Troubleshooting guide search
- FAQ and knowledge base systems
- Support ticket assistance

## 🚀 Getting Started

Follow the cells below to:
1. Install required dependencies and import libraries
2. Create a vector store with sample documents
3. Configure an agent with file search capabilities
4. Query the agent and receive grounded responses

Let's build an intelligent document-aware assistant! 📖✨

In [1]:
! pip install -r ../../../Installation/requirements.txt -U

  Cloning https://github.com/microsoft/agent-framework.git (to revision main) to /private/var/folders/h1/d_gywhr90l1g98mqlt6wwrkh0000gn/T/pip-install-k8betlpk/agent-framework-core_cefff85a510e47a795a3e0c54d9f8b4a
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/agent-framework.git /private/var/folders/h1/d_gywhr90l1g98mqlt6wwrkh0000gn/T/pip-install-k8betlpk/agent-framework-core_cefff85a510e47a795a3e0c54d9f8b4a
  Resolved https://github.com/microsoft/agent-framework.git to commit 9e3b2fa09a916c5a90dc4177a3b5e245f7ed2398
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/microsoft/agent-framework.git (to revision main) to /private/var/folders/h1/d_gywhr90l1g98mqlt6wwrkh0000gn/T/pip-install-k8betlpk/agent-framework-azure-ai_7086b950ab614513a430dd2a53d137e4
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/agen

In [2]:
import os

from azure.identity.aio import AzureCliCredential
from azure.ai.projects.aio import AIProjectClient
from dotenv import load_dotenv

from azure.ai.agents.models import FilePurpose,FileSearchTool
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent,HostedFileSearchTool,HostedVectorStoreContent

## 📦 Import Required Libraries

Import Azure AI libraries for project management, agent creation, and file search capabilities.

In [3]:
load_dotenv()

True

## 🔧 Load Environment Variables

Load Azure AI Foundry configuration from the `.env` file.

In [4]:
async def create_vector_store(client: AIProjectClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = './document.md'
    file = await client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

## 🗂️ Define Vector Store Creation Function

This async function handles document upload and vector store creation:
1. Uploads the document file to Azure AI Foundry
2. Creates a vector store with the uploaded file
3. Returns the file ID and vector store content for agent use

In [5]:
async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):

        file_id, vector_store = await create_vector_store(client)
        file_search = FileSearchTool(vector_store_ids=[vector_store.vector_store_id])
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="PythonRAGAgent",
            instructions="""
                    You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                    - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                    "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                    - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                    - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                    - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                    - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                    You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                    """,
            tools = file_search.definitions,
            tool_resources= file_search.resources
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)


        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
            ) as agent:
                    

            print("Agent created. You can now ask questions about the uploaded document.")

            query = "Can you explain Contoso's travel insurance coverage?"

            async for chunk in agent.run_stream(query, tools=HostedFileSearchTool(inputs=vector_store)):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

AttributeError: 'AgentsOperations' object has no attribute 'files'

## 🚀 Create and Run the RAG Agent

This cell performs the complete RAG workflow:
1. **Authentication**: Uses Azure CLI credentials for secure access
2. **Vector Store Setup**: Creates a searchable index from the document
3. **Agent Creation**: Configures an agent with file search capabilities
4. **Query Execution**: Streams the agent's response to a document-based question

The agent is instructed to:
- Only answer using information from the uploaded document
- Clearly state when information is not available in the document
- Provide explicit citations to source content